#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


In [4]:
df_ks.shape

(150000, 14)

Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

## Question 0

### Nettoyer les données

In [5]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

En voulant convertir les goals en float on a une erreur qui  nous dit que la valeur '2014-04-17' ne peut être convertis en float, donc il y a une ligne ou la valeur est une date au lieu d'un nombre

In [6]:
try:
    df_ks['goal'].astype(float)
except ValueError as e:
    print(e)

could not convert string to float: '2014-04-17'


On trouve la ligne correspondante et on constate qu'il y a un décalage des valeurs

In [7]:
df_ks.loc[df_ks['goal']=='2014-04-17']

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
66141,85964225,Debut Album from Michael Adam,Grace is Leaving,Indie Rock,Music,USD,2014-04-17,700.0,2014-04-02 21:56:35,850.0,successful,32,US,850.0


In [8]:
df_ks.iloc[66141]

ID                                        85964225
name                Debut Album from Michael Adam 
category                          Grace is Leaving
main_category                           Indie Rock
currency                                     Music
deadline                                       USD
goal                                    2014-04-17
launched                                     700.0
pledged                        2014-04-02 21:56:35
state                                        850.0
backers                                 successful
country                                         32
usd pledged                                     US
usd_pledged_real                               850
Name: 66141, dtype: object

En supposant qu'il n'y a que cette ligne qui pose problème et en retrouvant la page Kickstarter originelle on peut ré-insérer les données correctement à la main

In [9]:
#https://www.kickstarter.com/projects/michaeladam/debut-album-from-michael-adam-grace-is-leaving/posts?lang=fr
df_ks.iloc[66141] = pd.Series({ 'ID': 85964225,
            'name': 'Debut Album from Michael Adam Grace is Leaving',
            'category' : 'Indie Rock',
            'main_category' : 'Music',
            'currency' : 'USD',
            'deadline' : '2014-04-17',
            'goal' : 700.0,
            'launched': '2014-04-02 21:56:35',
            'pledged' :  850.0,
            'state' : 'successful',
            'backers' : 32,
            'country' : 'US',
            'usd pledged' : 850.0,
            'usd_pledged_real': 850.0
          })

On vérifie si en modifiant la ligne cela résout le problème

In [10]:
df_ks.to_csv('corrected_dataframe.csv', index = False, header=True)
df_test = pd.read_csv('corrected_dataframe.csv')
df_test.head(2)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.0,1145.0
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.0,1851.0


In [11]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


On convertit les dates en datetime, les sommes en float et les décomptes en entier (backers)

In [12]:
df_ks['goal']=df_ks['goal'].astype(float)
df_ks['pledged']=df_ks['pledged'].astype(float)
df_ks['usd pledged']=df_ks['usd pledged'].astype(float)
df_ks['usd_pledged_real']=df_ks['usd_pledged_real'].astype(float)


df_ks['backers']=df_ks['backers'].astype(int)

df_ks['deadline']=pd.to_datetime(df_ks['deadline'])
df_ks['launched']=pd.to_datetime(df_ks['launched'])

On regarde la présence de Nan

In [13]:
df_ks.isna().sum()

ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64

In [14]:
df_ks[df_ks['name'].isna()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
111797,1848699072,NaN,Narrative Film,Film & Video,USD,2012-02-29,200000.0,2012-01-01 12:35:31,100.0,failed,1,US,100.000000,100.000000
119500,634871725,NaN,Video Games,Games,GBP,2013-01-06,2000.0,2012-12-19 23:57:48,196.0,failed,12,GB,317.728436,316.047472


On drop les lignes où les names sont des Nan car on peut pas calculer/retrouver l'information

In [15]:
df_ks=df_ks.drop(df_ks[df_ks['name'].isna()].index)

On a 1482 lignes où la somme promise en USD n'est pas définie mais ce n'est pas trop grave car cette colonne n'est pas nécessaire pour répondre aux questions

In [16]:
df_ks[df_ks['usd pledged'].isna()][0:2]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
97,2146928537,"Crimson Apple, Debut Album ""Hello""",Music,Music,USD,2015-08-27,8000.0,2015-07-27 13:53:12,8805.0,undefined,0,"N,0""",NaN,8805.0
315,1927293317,Dillon King's Debut Album - Everything Changes,Music,Music,USD,2015-04-24,3500.0,2015-03-10 03:14:57,171.0,undefined,0,"N,0""",NaN,171.0


On vérifie si parmis les variables catégorielles il y a des valeurs étranges



In [17]:
df_ks['category'].unique()[0:5]

array(['Rock', 'Fashion', 'Tabletop Games', 'Art Books', 'Sculpture'],
      dtype=object)

In [18]:
df_ks['main_category'].unique()

array(['Music', 'Fashion', 'Games', 'Publishing', 'Art', 'Technology',
       'Film & Video', 'Journalism', 'Comics', 'Theater', 'Dance',
       'Design', 'Crafts', 'Food', 'Photography'], dtype=object)

In [19]:
df_ks['currency'].unique()

array(['USD', 'CAD', 'HKD', 'EUR', 'GBP', 'NOK', 'AUD', 'CHF', 'SEK',
       'NZD', 'MXN', 'DKK', 'SGD', 'JPY'], dtype=object)

In [20]:
df_ks['country'].unique()

array(['US', 'CA', 'HK', 'BE', 'GB', 'NO', 'NL', 'N,0"', 'AU', 'CH', 'SE',
       'AT', 'NZ', 'MX', 'FR', 'DK', 'IT', 'ES', 'DE', 'IE', 'SG', 'LU',
       'JP'], dtype=object)

On remarque un code pays inconnu N,0", cependant il n'estpas possible de savoir à quel pays il appartient car les devises utilisées avec ce code pays sont multiples 

In [21]:
df_ks.groupby(['country','currency']).size()

country  currency
AT       EUR            256
AU       AUD           3122
BE       EUR            234
CA       CAD           5889
CH       CHF            309
DE       EUR           1678
DK       DKK            431
ES       EUR            880
FR       EUR           1173
GB       GBP          13203
HK       HKD            251
IE       EUR            319
IT       EUR           1143
JP       JPY             12
LU       EUR             25
MX       MXN            720
N,0"     AUD             45
         CAD             93
         CHF              1
         DKK              5
         EUR             67
         GBP            185
         NOK              5
         NZD             13
         SEK             11
         USD           1057
NL       EUR           1120
NO       NOK            275
NZ       NZD            556
SE       SEK            713
SG       SGD            230
US       USD         115977
dtype: int64

De plus les lignes où le code pays est N,0" semblent être les mêmes que celles où usd pleged n'est pas défini

In [22]:
df_ks.groupby(['country']).size()

country
AT         256
AU        3122
BE         234
CA        5889
CH         309
DE        1678
DK         431
ES         880
FR        1173
GB       13203
HK         251
IE         319
IT        1143
JP          12
LU          25
MX         720
N,0"      1482
NL        1120
NO         275
NZ         556
SE         713
SG         230
US      115977
dtype: int64

On remarque aussi qu il y a 1217 lignes où n'y a aucun backers mais oú l'argent récolté est supérieur à zéro, cependant cela semble encore être les lignes ou le pays est N,0"

In [23]:
df_ks[(df_ks['backers']==0) & (df_ks['pledged']>0.0)].shape

(1217, 14)

In [24]:
df_ks[(df_ks['backers']==0) & (df_ks['pledged']>0.0)][0:3]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
97,2146928537,"Crimson Apple, Debut Album ""Hello""",Music,Music,USD,2015-08-27,8000.0,2015-07-27 13:53:12,8805.0,undefined,0,"N,0""",NaN,8805.0
315,1927293317,Dillon King's Debut Album - Everything Changes,Music,Music,USD,2015-04-24,3500.0,2015-03-10 03:14:57,171.0,undefined,0,"N,0""",NaN,171.0
502,97037549,The UNC Tarpeggios Album Project,Music,Music,USD,2015-04-30,5000.0,2015-03-29 19:13:28,7000.0,undefined,0,"N,0""",NaN,7000.0


In [25]:
df_ks = df_ks.drop(df_ks[df_ks['country']=='N,0"'].index)

On remarque en droppant les lignes où 'country' vaut N,0" on résout nos problèmes précédents

In [26]:
df_ks[(df_ks['backers']==0) & (df_ks['pledged']>0.0)]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
135961,950767056,Sins Of A Son,Documentary,Film & Video,USD,2010-09-04,3500.0,2010-07-02 23:41:15,1.0,failed,0,US,1.0,1.0


In [27]:
df_ks.isna().sum()

ID                  0
name                0
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
dtype: int64

In [28]:
df_ks = df_ks.drop( df_ks[(df_ks['backers']==0) & (df_ks['pledged']>0.0)].index)

In [29]:
(1-df_ks.shape[0]/150000)*100 # % données supprimées

0.990000000000002

### Importer les données

In [30]:
import json

In [56]:
result = df_ks.to_json(orient="records",date_format="iso")
parsed = json.loads(result)

In [57]:
parsed[0]

{'ID': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16T00:00:00.000Z',
 'goal': 2000.0,
 'launched': '2011-08-17T06:31:31.000Z',
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

In [49]:
database = client['exercices']
collection = database['kickstarter']

In [50]:
collection.insert_many(parsed)

In [51]:
collection.find_one()

{'_id': ObjectId('5fc50aa6996ce0eb6678787f'),
 'ID': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16T00:00:00.000Z',
 'goal': 2000.0,
 'launched': '2011-08-17T06:31:31.000Z',
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}


- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

## Question 1  

In [62]:
list(collection.find().sort([('pledged',-1)]).limit(5))

[{'_id': ObjectId('5fc50aa7996ce0eb6679a09d'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30T00:00:00.000Z',
  'goal': 50000.0,
  'launched': '2014-07-08T10:14:37.000Z',
  'pledged': 13285226.36,
  'state': 'successful',
  'backers': 62642,
  'country': 'US',
  'usd pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('5fc50aa7996ce0eb6679fac7'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30T00:00:00.000Z',
  'goal': 1000000.0,
  'launched': '2016-05-24T15:49:52.000Z',
  'pledged': 12779843.49,
  'state': 'successful',
  'backers': 66673,
  'country': 'US',
  'usd pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('5fc50aa7996ce0eb6679acb5'),
  'ID': 2111

## Question 2

In [68]:
list(collection.aggregate([
    {"$match" : {"state" : "successful"}},
    {"$count" : "nb_successful"}
]))

[{'nb_successful': 52999}]

## Question 3

In [81]:
list(collection.aggregate([
    {"$group" : { "_id" : "$category" , "count" : {"$sum":1}}},
   
]))

[{'_id': 'Video Games', 'count': 4797},
 {'_id': 'Calendars', 'count': 112},
 {'_id': 'Translations', 'count': 76},
 {'_id': 'Fiction', 'count': 3703},
 {'_id': 'Academic', 'count': 367},
 {'_id': 'Thrillers', 'count': 302},
 {'_id': "Farmer's Markets", 'count': 175},
 {'_id': 'Wearables', 'count': 508},
 {'_id': 'Comics', 'count': 1931},
 {'_id': 'Food', 'count': 4612},
 {'_id': 'Technology', 'count': 2689},
 {'_id': 'Periodicals', 'count': 517},
 {'_id': 'Documentary', 'count': 6496},
 {'_id': 'Action', 'count': 282},
 {'_id': 'Interactive Design', 'count': 146},
 {'_id': 'Festivals', 'count': 313},
 {'_id': 'Robots', 'count': 237},
 {'_id': 'Performance Art', 'count': 855},
 {'_id': 'Sound', 'count': 255},
 {'_id': 'Fabrication Tools', 'count': 96},
 {'_id': 'Narrative Film', 'count': 2097},
 {'_id': 'Vegan', 'count': 229},
 {'_id': 'Country & Folk', 'count': 1790},
 {'_id': 'Woodworking', 'count': 433},
 {'_id': 'Television', 'count': 401},
 {'_id': 'Fantasy', 'count': 132},
 {'_id

## Question 4

In [89]:
#https://docs.mongodb.com/manual/reference/method/Date/index.html

list(collection.aggregate([
    {"$match" : {"country" : "FR" , "launched":{"$lt": 'new Date("2016-00-00T00:00:00Z")'}}},
    {"$count" : "nb_FR_2016"}
]))

[{'nb_FR_2016': 1173}]

## Question 5

In [93]:
next(collection.aggregate([
    {"$match" : {"country" : "US" ,"currency":"USD", "goal":{"$gt": 200000}}}
    
]))

{'_id': ObjectId('5fc50aa6996ce0eb66787920'),
 'ID': 866634482,
 'name': 'A CALL TO ADVENTURE',
 'category': 'Film & Video',
 'main_category': 'Film & Video',
 'currency': 'USD',
 'deadline': '2012-09-14T00:00:00.000Z',
 'goal': 287000.0,
 'launched': '2012-08-13T23:14:02.000Z',
 'pledged': 1465.0,
 'state': 'failed',
 'backers': 11,
 'country': 'US',
 'usd pledged': 1465.0,
 'usd_pledged_real': 1465.0}

## Question 6 

In [94]:
collection.create_index([("name",  "text")])

'name_text'

In [95]:
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [100]:
next(collection.find( { "$text": { "$search": "Sport" } } ))

{'_id': ObjectId('5fc50aa7996ce0eb6679260b'),
 'ID': 1875366029,
 'name': 'Sport Smart. A New Genre of Sports TV. Sport Fans Unite!!!',
 'category': 'Webseries',
 'main_category': 'Film & Video',
 'currency': 'USD',
 'deadline': '2011-01-15T00:00:00.000Z',
 'goal': 4500.0,
 'launched': '2010-12-16T08:40:02.000Z',
 'pledged': 25.0,
 'state': 'failed',
 'backers': 1,
 'country': 'US',
 'usd pledged': 25.0,
 'usd_pledged_real': 25.0}